# Component Selection for Cardiac vs. Breathing Separation
This notebook performs kernel PCA using the chosen sigma value (from the previous notebook)
and displays the time series, FFT, and PSD for each component.
You can also display cine reconstructions for individual components.
Based on these outputs, decide which component indices are cardiac (i.e., do not contain significant respiratory motion).
Record the list of selected components for use in the final pipeline.

### Loading packages and data

In [ ]:
import yaml
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
import utils.data_ingestion as di
import utils.pca as pca
import utils.gif as gif
import utils.reconstruction as recon

def load_config(config_file="config.yaml"):
    with open(config_file, "r") as f:
        return yaml.safe_load(f)

config = load_config()

# Set file paths and parameters
twix_file = config["data"]["twix_file"]
dicom_folder = config["data"]["dicom_folder"]
scans = di.read_twix_file(twix_file, include_scans=[-1], parse_pmu=False)
kspace = di.extract_image_data(scans)
n_phase_encodes_per_frame = kspace.shape[0] // config["data"]["n_frames"]
extended_phase_lines = config["data"]["extended_pe_lines"]
row_offset = config["data"]["offset"]

decomp_method = config["processing"]["decomposition_method"].lower()
n_components = config["processing"].get("n_components", None)

# Get frame rate from DICOM files.
framerate, _ = di.get_dicom_framerate(dicom_folder)

### Performing kernel PCA and displaying components

In [ ]:
if decomp_method == "kernel_pca":
    sigma = config["processing"]["sigma"]
    model, X_trans, frame_shape, orig_feature_dim = pca.perform_kernel_pca(
        kspace, n_phase_encodes_per_frame, kernel="rbf", sigma=sigma, n_components=n_components
    )
elif decomp_method == "ica":
    model, X_trans, frame_shape, orig_feature_dim = pca.perform_ica(
        kspace, n_phase_encodes_per_frame, n_components=n_components
    )
elif decomp_method == "pca":
    model = pca.perform_pca(kspace, n_phase_encodes_per_frame)
    frame_shape = model[-1]
    X_trans = None
    orig_feature_dim = model[3].shape[0]
else:
    raise ValueError(f"Unknown decomposition method: {decomp_method}")

# Plot components: if transformed data exists, use that; otherwise use PCA plotting.
if X_trans is not None:
    pca.plot_components_time_series_and_fft(X_trans, sampling_rate=framerate, n_components=10)
else:
    pca.plot_pca_time_series_and_fft(model[0], model[1], sampling_rate=framerate, n_components=10)

# Reconstruct and display cine movies for each component.
for comp in range(10):
    recon_component = pca.reconstruct_kspace_from_components(
        decomp_method, model, X_trans, comp, frame_shape, orig_feature_dim=orig_feature_dim
    )
    recon_cine = recon.direct_ifft_reconstruction(recon_component, extended_phase_lines, row_offset, True)
    print(f"Displaying reconstruction using component {comp}:")
    display(gif.display_images_as_gif(recon_cine, interval=100, notebook=True))
    
# Decide which components represent cardiac motion and record those indices for the final pipeline.